<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-изучение-данных" data-toc-modified-id="Загрузка-и-изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и изучение данных</a></span></li><li><span><a href="#Построение-модели-по-первоначальным-данным" data-toc-modified-id="Построение-модели-по-первоначальным-данным-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Построение модели по первоначальным данным</a></span></li><li><span><a href="#Проверка-влияния-преобразования-данных-на-качество-линейной-регрессии" data-toc-modified-id="Проверка-влияния-преобразования-данных-на-качество-линейной-регрессии-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка влияния преобразования данных на качество линейной регрессии</a></span></li><li><span><a href="#Обоснование-математического-аппарата-шифрования-данных" data-toc-modified-id="Обоснование-математического-аппарата-шифрования-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обоснование математического аппарата шифрования данных</a></span></li><li><span><a href="#Создание-алгоритма-шифрования-данных" data-toc-modified-id="Создание-алгоритма-шифрования-данных-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Создание алгоритма шифрования данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error as mse
import random

## Загрузка и изучение данных

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

Выведем 5 случайных строк датасета

In [3]:
data.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
1411,1,31.0,21600.0,0,0
1289,0,21.0,54800.0,1,0
3386,1,29.0,41800.0,1,0
605,1,31.0,52000.0,2,0
2248,0,39.0,33800.0,0,0


Выедем общую информацию по датасету

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет, все типы данных корректны

Проверим наличие дубликатов

In [5]:
data.duplicated().sum()

153

В датасете есть дубликаты. Для решения задачи повышения точности обученной модели их следовало бы удалить, но в данном случае они не помешают сравнению моделей, обученных на оригинальных и преобразованных данных.

Выделим в датасете признаки и целевую переменную

In [6]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Разделим датасет на обучающую и тестовую выборки в пропорции 3:1

In [7]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.25)

## Построение модели по первоначальным данным

In [8]:
base_model = LinearRegression()
base_model.fit(train_features, train_target)
predicted = base_model.predict(test_features)
r2_score(test_target, predicted)

0.4123336606937037

## Проверка влияния преобразования данных на качество линейной регрессии

Графики предсказаний для линейной регрессии задаются уравнением:

$$ \large y=\omega x+\omega _0, $$

где $y$ - предсказания модели, $\omega$ - матрица весов линейной регрессии, $x$ - матрица признаков, $\omega_0\$ - смещение

Задача обучения линейной регрессии состоит в том, чтобы добиться минимальной метрики MSE между предсказаниями на обучающей выборке и целевой переменной обучающей выборки. Этого можно добиться тщательным подбором матрицы весов $\omega$, то есть:

$$ \large \omega = arg \: \underset{\omega}{min} \: MSE(X\omega, y) $$

Минимум MSE достигается тогда, когда верно равенство:

$$ \large \omega = (X^TX)^{-1}X^Ty    $$

Преобразуем его:

$$ \large (X^TX)^{-1}X^Ty=X^{-1} \: [\overset{E}{(X^T)^{-1} X^T}] \: y = X^{-1}Ey = X^{-1}y $$

Получаем:

$$ \large \omega = X^{-1}y $$

Пусть матрица признаков умножается на некоторую матрицу $M$, тогда выражение (3) приобретает вид:

$$ \large \omega' = ((XM)^T(XM))^{-1}(XM)^Ty    $$

введем замену:

$$ \large A=XM $$

тогда (6) преобразуется в:

$$ \large \omega' = (A^TA)^{-1}A^Ty $$

Преобразуем (8) его по аналогии с (4):

$$ \large (A^TA)^{-1}A^Ty = (A)^{-1} \: [\overset{E}{(A^T)^{-1} A^T}] \: y = A^{-1}E y = A^{-1} y $$

Из (8) и (9) получаем:

$$ \large \omega' = A^{-1} y $$

Проделаем замену, обратную (7) и преобразуем выражение (10)

$$ \large \omega' = (XM)^{-1} y = M^{-1} X^{-1} y  $$

Учитывая (5) и (11), мы пришли к равенству:

$$ \large \omega' = M^{-1}\omega $$

Это означает, что переход от первоначальной матрицы весов к преобразованной" возможен путем умножения ее на обратимую матрицу.

***Вывод***

При умножении матрицы признаков на обратимую матрицу качество линейной регрессии, заново обученной по новому набору данных, не меняется.

Второе доказательство

$$ \large \omega' = ((XM)^T(XM))^{-1}(XM)^Ty    $$

$$ \large \omega' = (XM)^{-1} ((XM)^T)^{-1} (XM)^T y    $$

$$ \large \omega' = (XM)^{-1}  \: [\overset{E}{((XM)^T)^{-1} (XM)^T}] \: y    $$

$$ \large \omega' = (XM)^{-1} \: E \: y    $$

$$ \large \omega' = M^{-1} X^{-1} y    $$

$$ \large \omega' = M^{-1}  \: [\overset{\omega}{X^{-1} y}]    $$

$$ \large \omega' = M^{-1} \omega    $$

## Обоснование математического аппарата шифрования данных

Для хранения личные данные пользователей можно зашифровать. Для этого матрицу признаков можно умножить на обратимую матрицу-ключ, и потом использовать эти данные в линейной регрессии без потери качества модели. Также по матрицу-ключу возможно восстановить исходные данные.

Проведем вычисления, подтверждающие это. Умножим матрицу признаков на обратимую матрицу. Есть два способа сделать это: умножить признаки на матрицу слева или справа. В зависимости от выбранного способа будет меняться размерность матрицы.

$$ \large \overset{(3750, \: 4)}{X} \: \overset{(4, \: 4)}{M} = \overset{(3750, \: 4)}{X'} $$
$$ \large \overset{(3750, \: 3750)}{M} \: \overset{(3750, \: 4)}{X} = \overset{(3750, \: 4)}{X'} $$

У обоих способов есть недостатки. В первом случае размерность ключа мала, так как зависит от числа признаков, и это небезопасно. Во втором случае наблюдается наоборот неконтролируемый рост сложности, потому что с каждым новым клиентом придется менять матрицу-ключ. В рамках учебной задачи остановимся на первом варианте.

## Создание алгоритма шифрования данных

Напишем функцию, подбирающую данным ключ подходящей размерности, и возвращающую преобразованные датасеты вместе с ключом.

In [9]:
def encoded_data(data1, data2):
    key_matrix = [[random.randint(0,999) for i in range(data1.shape[1])] for j in range(data1.shape[1])]
    try:
        reversed_matrix = np.linalg.inv(key_matrix)
    except:
        pass
    return data1@key_matrix, data2@key_matrix, key_matrix

Преобразуем данные при помощи шифратора

In [10]:
encoded_train_features, encoded_test_features, key = encoded_data(train_features, test_features)

Обучим модель линейной регрессии на новых данных

In [11]:
model = LinearRegression()
model.fit(encoded_train_features, train_target)
predicted = model.predict(encoded_test_features)
r2_score(test_target, predicted)

0.4123336606937128

Точность модели не изменилась

Проверим обратимость матрицы-ключа. Выведем ключ и обратную к нему матрицу

In [12]:
pd.DataFrame(key)

,0,1,2,3
0,875,78,150,297
1,509,228,704,126
2,302,848,240,948
3,546,838,64,772


In [13]:
pd.DataFrame(np.linalg.inv(key))

,0,1,2,3
0,0.000840,0.000100,-0.001084,0.000991
1,-0.001936,0.000919,-0.002446,0.003599
2,-0.000254,0.001260,0.000978,-0.001309
3,0.001529,-0.001173,0.003341,-0.003204


## Вывод

* Линейная регрессия построена на матричных операциях
* Математически и практически доказано, что умножение матрицы признаков на обратимую матрицу не меняет качество линейной регрессии
* Это свойство можно использовать для шифрования данных
* Умножение ключа слева и справа на матрицу устроено по-разному и имеет разные степени сложности и безопасности